In [4]:
from bs4 import BeautifulSoup
import requests
import time
import os
import re

## 2.Interact with the Page-Sorting:
Initially, the listings might be sorted by “newest” first.  Try changing the sorting order to “oldest” first by interacting with the page’s UI.

Observe any changes in the URL after you change the sorting order back and forth.

Can you trigger the sorting change directly by modifying only the URL in your browser’s address bar?  If so, how?

Explain what type of request is made when you change the sort order (GET or POST).

What is the variable in the URL associated with sorting?

The url changes as we change the filter, and we can triger the sorting directly by modifying the URL.
The NEW url is 'https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0', we can manipulate the sorting by changing whats after the 'sort = ', this is whats associated with url.
Changing the sort order by modifying the URL and pressing Enter typically makes a GET request, because its used to retrive data from a specified resource.

## 3. Interact with the Page-Pagination:
Craigslist paginates listings, typically displaying a limited number of items per page (120 for me).

Navigate to the second and third pages of results and observe the changes in the URL.

Exploration Task:

Determine how to move between pages by only changing the URL.  What part of the URL changes as you navigate through different pages?

This task will help you understand how pagination works on Craigslist and how you can programmatically access different pages of listings.

Identify the variable associated with page changes.  How does altering this variable in the URL affect the page you’re viewing?

Explain.

I noticed that the url changes at the end as a  scroll down on the page and as I go to a different page.
on the first page: https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0.
Second page: https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~1~0.
The part thats associated with page and results are 0~0 and 1~0. The number before ~ represents page number -1. The number following ~ signifies the proportion of items displayed on the screen from the current page.

I can go to different pages by changing the number in  front of ~, and I can change the number following ~ to scroll down or up. For example. I can change the url to https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~4~0 if I want to see the top of 3rd page.

## 4. Fetch Listing URLs:
Use `requests` to access the first page of the “free” section, ordered “newest” first.

Deploy `BeautifulSoup` to parse the HTML content.

Identify the structure that holds the links to individual listing pages.  What selector do you choose to grab the link?

Can you identify one more possible selection method to retrieve the link to the individual listing?  Explain.

Extract the first 250 unique listing URLs and save them to a list.  Consider the pagination feature of Craigslist to navigate through pages.  Explain your strategy.

Print the list to screen.

In [12]:
URL = 'https://sfbay.craigslist.org/search/zip#search=1~gallery~0~0'
headers = {'User-Agent':'Mozilla/5.0'}
page = requests.get(URL, headers= headers)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="craigslist" property="og:site_name"/>
  <meta content="preview" name="twitter:card"/>
  <meta content="SF bay area free stuff - craigslist" property="og:title"/>
  <meta content="SF bay area free stuff - craigslist" name="description"/>
  <meta content="SF bay area free stuff - craigslist" property="og:description"/>
  <meta content="https://sfbay.craigslist.org/search/zip" property="og:url"/>
  <title>
   SF bay area free stuff - craigslist
  </title>
  <link href="https://sfbay.craigslist.org/search/zip" rel="canonical"/>
  <link href="https://sfbay.craigslist.org/search/zip" hreflang="x-default" rel="alternate"/>
  <link href="/favicon.ico" id="favicon" rel="icon">
   <script id="ld_searchpage_data" type="application/ld+json">
    {"description":"Free Stuff in SF Bay Area","@type":"Se

I found that all a tags that contain listings' urls are inside of li class="cl-static-search-result". I chose this selector to find all listing urls.
Another selector can be the div class = 'title', and find the previous sibling a.

In [24]:
# Find all <li> tags with the class 'cl-static-search-result'
listing_items = soup.find_all('li', class_='cl-static-search-result')
# create an empty list to later store all the urls
listing_urls = set()

for item in listing_items:
    a_tag = item.find('a', href=True)  #find the a tag within each listing_item
    listing_urls.add(a_tag['href']) # add into the list once found


In [29]:
listing_urls_250 = list(listing_urls)[:250] # Choose the first 250

for listing in listing_urls_250:
    print(listing)

https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-bag-of-takeout/7714724056.html
https://sfbay.craigslist.org/eby/zip/d/hayward-ikea-plate-mats/7714138516.html
https://sfbay.craigslist.org/nby/zip/d/windsor-foam/7714783645.html
https://sfbay.craigslist.org/nby/zip/d/larkspur-double-bed-mattress-and-bread/7714771047.html
https://sfbay.craigslist.org/eby/zip/d/oakland-aquarium-test-kit/7714843393.html
https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-laclede-cable-tire/7713262861.html
https://sfbay.craigslist.org/pen/zip/d/menlo-park-free-metal-file-cabinets/7710526505.html
https://sfbay.craigslist.org/sby/zip/d/san-jose-moving-daybed-and-desk/7714870304.html
https://sfbay.craigslist.org/pen/zip/d/south-san-francisco-chicken-scratch/7714851829.html
https://sfbay.craigslist.org/sfc/zip/d/san-francisco-acrylic-shelf/7714624891.html
https://sfbay.craigslist.org/pen/zip/d/daly-city-white-sofa-console-table/7714847253.html
https://sfbay.craigslist.org/sby/zip/d/sunnyvale-free-

## 5.Save HTML Pages:
For each of the 250 listing URLs, use `requests` to fetch the listing page.

Save each HTML content to a separate file on disk.  Use each listing’s ID to organize files in a way that makes them easily identifiable (e.g., save listing ID 7713901653 to file “7713901653.html”).

In [35]:
for listing in listing_urls_250:
    response = requests.get(listing) #get response from each listing's url
    listing_id = listing.split('/')[-1].split('.')[0]  #extract the id
    listing_files = os.path.join('/DDR', f"{listing_id}.html") # save the file and name it as required
    
    with open(listing_files, 'w', encoding='utf-8') as file:
        file.write(response.text) #write text of each listing page to corresponding file
    
    print(f"Saved {listing_id}.html") # print it once as one loop is completed
    time.sleep(5) #sleep 5 seconds before next loop

Saved 7714724056.html
Saved 7714138516.html
Saved 7714783645.html
Saved 7714771047.html
Saved 7714843393.html
Saved 7713262861.html
Saved 7710526505.html
Saved 7714870304.html
Saved 7714851829.html
Saved 7714624891.html
Saved 7714847253.html
Saved 7714835841.html
Saved 7714866747.html
Saved 7714790955.html
Saved 7714840319.html
Saved 7714858528.html
Saved 7712594016.html
Saved 7714840486.html
Saved 7714673824.html
Saved 7714763271.html
Saved 7714739763.html
Saved 7707358474.html
Saved 7714809240.html
Saved 7714620649.html
Saved 7714745012.html
Saved 7708570461.html
Saved 7713504460.html
Saved 7714851113.html
Saved 7714855414.html
Saved 7714795155.html
Saved 7714865577.html
Saved 7714864012.html
Saved 7707259091.html
Saved 7705565710.html
Saved 7705456929.html
Saved 7714826961.html
Saved 7714751706.html
Saved 7710401685.html
Saved 7714745745.html
Saved 7711966087.html
Saved 7713938867.html
Saved 7708083364.html
Saved 7714832903.html
Saved 7714836542.html
Saved 7714730260.html
Saved 7714

# Part 2

## 1. Read Saved HTML Files:
Write a script that reads each of the saved HTML files from the disk.

Your code could look similar to

Loop through each file in the directory

for filename in os.listdir(directory):

    Check if the file ends with .html

    if filename.endswith(".html"):

         Construct the full file path

        filepath = os.path.join(directory, filename)

         Read file to string

        with open(filepath, 'r', encoding='utf-8') as file:

            html = file.read()

         ... use html string ...

In [36]:
for filename in os.listdir('/DDR'):
    if filename.endswith(".html"):
        filepath = os.path.join('/DDR',filename)
        with open(filepath,'r',encoding='utf-8') as file:
            html = file.read()
        print(f"Read {filename}.")

Read 7714731668.html.
Read 7714755250.html.
Read 7714745745.html.
Read 7714751706.html.
Read 7712106559.html.
Read 7714842895.html.
Read 7713504460.html.
Read 7714640189.html.
Read 7711142472.html.
Read 7714033238.html.
Read 7714768144.html.
Read 7714646374.html.
Read 7714735168.html.
Read 7714847253.html.
Read 7714711513.html.
Read 7714772590.html.
Read 7712902779.html.
Read 7714666131.html.
Read 7714826961.html.
Read 7711197030.html.
Read 7714728310.html.
Read 7714716184.html.
Read 7710401685.html.
Read 7714781080.html.
Read 7714747562.html.
Read 7710526505.html.
Read 7708161021.html.
Read 7714826649.html.
Read 7714825659.html.
Read 7714822466.html.
Read 7710983170.html.
Read 7714815892.html.
Read 7714641003.html.
Read 7714721126.html.
Read 7713809947.html.
Read 7714829338.html.
Read 7708871157.html.
Read 7714854624.html.
Read 7714804485.html.
Read 7714869265.html.
Read 7714716795.html.
Read 7714672796.html.
Read 7714854612.html.
Read 7712196033.html.
Read 7714724056.html.
Read 77148

## 2. Extract Information:
For each HTML file, use `BeautifulSoup` to parse the file content.

Extract and print the following details:

Title: The title of the listing.

URL of first image (if an image exists):  The URL of the displayed image.  It can be found in the `src` attribute of `<img>`

Description: The full description text of the listing.

Post ID: Usually found at the bottom of the page or within the page's HTML structure.

Posted Date: The date when the listing was originally posted.

Last Updated Date: The date when the listing was last updated.



In [74]:

# read files using the same code above
for filename in os.listdir('/DDR'):
    if filename.endswith(".html"):
        filepath = os.path.join('/DDR',filename)

        with open(filepath,'r',encoding='utf-8') as file:
            html = file.read()
        print(f"Read {filename}.")

        # Have to add if else too all the needed variables in case it doesn't exist
        soup = BeautifulSoup(html, 'html.parser')
        title_tag = soup.select_one('#titletextonly')
        title = title_tag.text if title_tag else 'No Title Found'
        image_tag = soup.find('img')
        image = image_tag['src'] if image_tag else "No Image Found"

        description_tag = soup.select_one('#postingbody')
        if description_tag:
            qr_code_text = description_tag.find('div', class_='print-information print-qrcode-container')
            if qr_code_text:
                qr_code_text.extract()  # Remove the QR code text from the description, otherwise it will show up in the result
            description = description_tag.get_text(separator=' ', strip=True)
        else:
            description = 'No Description Found'


        post_id_tag = soup.select_one('div.postinginfos > p.postinginfo')
        # Select only the number after ':'
        post_id = post_id_tag.text.split(':')[-1].strip() if post_id_tag else "No Post ID Found"

        # Select the p that contains both posted date and updated date
        posted_p = soup.select('p.postinginfo.reveal')
        if posted_p:
            for i in posted_p:
                if re.search('posted', i.text):
                    posted_date = i.text.split(" ")[1] # split by space then get 2nd variable
                elif re.search('updated', i.text):
                    updated_date = i.text.split(" ")[1]
        else: 
            posted_date = "No Posted Date Found"
            updated_date = "No Updated Date Found"

       
        print(f"Title: {title}\n")
        print(f"Image: {image}\n")
        print(f"Description: {description}\n")
        print(f"Post ID: {post_id}\n")
        print(f"Posted Date: {posted_date}\n")
        print(f"Updated Date: {updated_date}\n")


Read 7714731668.html.
Title: FREE TEDDY BEARS & GALAXY ROSES FOR VALENTINE'S DAY TO SPREAD LOVE

Image: https://images.craigslist.org/00Y0Y_8bQG1tUwXSq_0cI0oc_600x450.jpg

Description: FREE TEDDY BEARS, VALENTINE PLUSH TOYS & GALAXY ROSES FOR VALENTINE'S DAY TO SPREAD LOVE on GARAGE SALE FRIDAY FEB 9, SATURDAY FEB 10 AT AND SUNDAY FEB 11 AT 249 SAGAMORE STREET IN SF FROM 10AM TO 1PM Happy Valentine's Day! See the video https://www.youtube.com/watch?v=PmlwGG1I75M

Post ID: 7714731668

Posted Date: 2024-02-05

Updated Date: 2024-02-05

Read 7714755250.html.
Title: Free tray with folding legs - structurally good.

Image: https://images.craigslist.org/00E0E_aRgKXFceWme_0t20CI_600x450.jpg

Description: Free tray with folding legs - structurally sound. Cover the top with a placemat and you would still be able to use it as a breakfast tray. Or drill holes and tighten legs to use as a tray to hold your dog’s food & water bowls? Help me re-use and save from landfill? Pick up in North Menlo Park

# Part 3: Automating Login on The Old Reader

 In this part, you will focus on automating the process of logging into https://theoldreader.comLinks to an external site..  This will involve understanding web authentication mechanisms, managing sessions, and verifying successful login programmatically.  This part is very similar to neopets which we did in class.  Please review the neopets code before attempting this part.

## 1. Creating and Verifying a The Old Reader Account
Account Creation:  Create an account on https://theoldreader.comLinks to an external site..  Use an email address and password that you are comfortable sharing with us.

Manual Login Verification: Before automating the login process, ensure you can manually log in to theoldreader.com with your new credentials.  This confirms that your account is active and your credentials are correct.

## 2. Exploring the Login Mechanism
Navigate to the login page of https://theoldreader.comLinks to an external site..

Use your browser’s developer tools to inspect the page, focusing on the <form> tag involved in the login process.

Document all `<input>` fields within the login form, paying special attention to their name attributes. These fields are crucial for submitting the login request programmatically.

In [75]:
headers = {'User-agent': 'Mozilla/5.0'}
url = 'https://theoldreader.com/users/sign_in'
time.sleep(5)
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
  <link href="https://fonts.googleapis.com/css?family=Montserrat:400,600" rel="stylesheet" type="text/css"/>
  <!-- Latest compiled and minified JavaScript -->
  <script src="https://code.jquery.com/jquery.js">
  </script>
  <script src="//netdna.bootstrapcdn.com/bootstrap/3.0.1/js/bootstrap.min.js">
  </script>
  <link href="https://fonts.googleapis.com/css?family=Source+Code+Pro" rel="stylesheet" type="text/css"/>
  <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,800" rel="stylesheet"/>
  <link href="//s.theoldreader.com/assets/reader/public-c7869a909c7b119a27fb646003828344.css" media="screen" rel="stylesheet" type="text/css">
   <link href="//s.theoldreader.com/assets/

In [78]:
all_inputs = soup.find_all('input')
for input in all_inputs:
    print(input)

<input name="utf8" type="hidden" value="✓"/>
<input name="authenticity_token" type="hidden" value="wZSFduCT+yZbXME5vXDcklXf4yoM8KMXFIei6TQmktw="/>
<input autocapitalize="off" autocorrect="off" autofocus="autofocus" class="form-control" id="user_login" name="user[login]" placeholder="Username/Email" size="30" spellcheck="false" type="text"/>
<input class="form-control" id="user_password" name="user[password]" placeholder="Password" size="30" type="password"/>
<input class="btn btn-primary btn-block" name="commit" type="submit" value="Sign In"/>


## 3. Analyzing Network Traffic for Login Request
With the network tab of your browser’s developer tools open, log in to the site again.

Identify the network request made when you submit the login form (GET or POST).  Explain why this method was chosen.

Carefully examine the payload that was submitted to the server during login.  Compare this payload to the `<form>` / `<input>` fields you previously analyzed.  Explain your observation.



The inputs' names scrapped from the URL mostly matches those on payload. But the authenticity_token values is not the same.
We need to use POST here because we are posting information to the url instead of getting information from the url

## 4. Automating the Login Process
Using Python and appropriate libraries like requests, simulate the login process.

Create a session object to maintain your login state across multiple requests.

Prepare a payload with your login credentials and other necessary form data identified from the login page and the network analysis.

Send a POST request to the login form’s action URL to log in, using the session object.

In [76]:
# Extract uthenticity_token required to login
input = soup.select_one('#new_user input[name = authenticity_token]')
authenticity_token = input.get('value')
print(authenticity_token)

time.sleep(5)

# An open session carries the cookies and allows you to make post requests
session = requests.session()

# We need to use POST here because we are posting information to the url instead of getting information from the url
res = session.post('https://theoldreader.com/users/sign_in', 
                        data = {
                                'utf8' : '%E2%9C%93', # We need to go en-coded view of the payload to get this
                                'authenticity_token' : authenticity_token,
                                'user[login]' : 'tu1242186698@gmail.com', #user name
                                'user[password]' : 'Tu1242186698**', # password
                                'commit': 'Sign In'},
                        timeout = 20)

# This will get us the Cookies.
cookies = session.cookies.get_dict()
print(cookies)

wZSFduCT+yZbXME5vXDcklXf4yoM8KMXFIei6TQmktw=
{'_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTc4ZjZmYmI2YWI5ZWQ3MDVjMzA5OGVjYjBmNWQyN2M0BjsAVEkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiEZI6DO5RIcp4yRK4DkkiIiQyYSQwNSRBMjhIUFpOZ29ObHpseHRIY2JQZGouBjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiEF9jc3JmX3Rva2VuBjsARkkiMUR2OWNwMHZNRi93VVljQVUrQm9ZMjNoNncvNmFtYXpMdU1oMFVQOGlpWUU9BjsARg%3D%3D--df795df185a400883a35bdf25a4e79cb486d4863', 'i_know_you': 'Ciny', 'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhGSOgzuUSHKeMkSuA5JIiIkMmEkMDUkQTI4SFBaTmdvTmx6bHh0SGNiUGRqLgY6BkVU--37a1090dafce6ae20e14bb62c9f55042e702edbf', 'signed_at': '1707278776'}


## 5. Verifying Successful Login
After attempting to log in, inspect the cookies saved in the session object to understand the information WhoScored.com stores on your computer.

Use the session object to access https://theoldreader.comLinks to an external site..

Verify successful login by checking for the presence of your user information that is only available when logged in.

In [81]:
print(cookies)

{'_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTc4ZjZmYmI2YWI5ZWQ3MDVjMzA5OGVjYjBmNWQyN2M0BjsAVEkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiEZI6DO5RIcp4yRK4DkkiIiQyYSQwNSRBMjhIUFpOZ29ObHpseHRIY2JQZGouBjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiEF9jc3JmX3Rva2VuBjsARkkiMUR2OWNwMHZNRi93VVljQVUrQm9ZMjNoNncvNmFtYXpMdU1oMFVQOGlpWUU9BjsARg%3D%3D--df795df185a400883a35bdf25a4e79cb486d4863', 'i_know_you': 'Ciny', 'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhGSOgzuUSHKeMkSuA5JIiIkMmEkMDUkQTI4SFBaTmdvTmx6bHh0SGNiUGRqLgY6BkVU--37a1090dafce6ae20e14bb62c9f55042e702edbf', 'signed_at': '1707278776'}


In [87]:

time.sleep(5) # 5s

# Get the page from profile's URL and remain cookie
page2 = requests.get('https://theoldreader.com/profile/923a0cee5121ca78c912b80e', cookies=cookies) # or explicitly set cookies of the session (when not using session.xyz)
soup2 = BeautifulSoup(page2.content, 'html.parser')

# print(soup3);
# print();
elements = soup2.select_one('h1.username') # Scrap my user name
print(elements.text)

Ciny
